### Pre-process the Data

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2
import random

In [2]:
def random_brightness(img, factor):
    # assume input is color image
    # convert Red, Green, Blue to Hue, Saturation, Value
    # factor describes the range of brightness scaling: (1 - factor, 1 + factor)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    bright_factor = random.uniform(1. - factor, 1. + factor)
    #print("brightness", bright_factor)
    img_hsv = np.array(img_hsv, np.float)
    img_hsv[:,:,2] = np.minimum(255, img_hsv[:,:,2] * bright_factor)
    img_hsv = np.array(img_hsv, dtype = np.uint8)
    img_new = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    
    return img_new

In [3]:
def xy_translation(img, steer, dfactor):
    # assume input is color image
    # dfactor is the fraction of total pixels as translation range
    nrow, ncol, nch = img.shape
    tX = dfactor * ncol * random.uniform(-1., 1.)
    tY = dfactor * 0.3 * nrow * random.uniform(-1., 1.) # give vertical translation less freedom
    #print("right shift =", tX)
    #print("down shift =", tY)
    Mtrans = np.float32([[1,0,tX], [0,1,tY]])
    img_new = cv2.warpAffine(img, Mtrans,(ncol, nrow))
    steer_new = steer + tX * .004
    return img_new, steer_new

In [4]:
def crop_resize(img, new_row, new_col):
    nrow, ncol, nch = img.shape
    # crop out img[y1:y2, x1:x2]
    # y1 = 0.3 * nrow: leave out the top 0.3 of image
    # y2 = nrow - 25: leave out the bottom 25 rows
    img_new = img[int(nrow * 0.3) : nrow - 25, 0 : ncol]
    img_new = cv2.resize(img_new,(new_col,new_row), interpolation=cv2.INTER_AREA)
    return img_new

In [5]:
data_dir = "./data/"
driving_log = pd.read_csv(data_dir + "driving_log.csv")

In [6]:
n_entries, n_feats = driving_log.shape

In [7]:
driving_log.head(3)

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011


In [8]:
driving_log.tail(3)

,center,left,right,steering,throttle,brake,speed
8033,IMG/center_2016_12_01_13_46_38_846.jpg,IMG/left_2016_12_01_13_46_38_846.jpg,IMG/right_2016_12_01_13_46_38_846.jpg,0.0,0.0,0.0,1.388364
8034,IMG/center_2016_12_01_13_46_38_922.jpg,IMG/left_2016_12_01_13_46_38_922.jpg,IMG/right_2016_12_01_13_46_38_922.jpg,0.0,0.0,0.0,1.377208
8035,IMG/center_2016_12_01_13_46_38_947.jpg,IMG/left_2016_12_01_13_46_38_947.jpg,IMG/right_2016_12_01_13_46_38_947.jpg,0.0,0.0,0.0,1.374433


In [9]:
steer_correct = {'left': -0.25, 'center' : 0., 'right' : 0.25}

In [10]:
new_row = 66
new_col = 200

In [11]:
def preprocess_train(entry):
    icamera = np.random.randint(3)
    cam_pos = list(steer_correct.keys())[icamera]
    filename = entry[cam_pos].strip()
    steering = float(entry['steering']) + float(steer_correct[cam_pos])
    img = cv2.imread(data_dir + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    br_img = random_brightness(img, 0.5)
    tr_img, steer_new = xy_translation(br_img, steering, 0.2)
    cr_img = crop_resize(tr_img, new_row, new_col)
    
    flip_ornot = np.random.randint(2)
    if flip_ornot == 1:
        image = cv2.flip(cr_img, 1)    #1 means flip around y-axis
        steer_new *= -1
    else:
        image = cr_img
        
    image = np.array(image)    # convert to numpy arrays for Keras
    
    return image, steer_new

In [12]:
def preprocess_valid(entry):
    filename = entry['center'].strip()
    steering = float(entry['steering'])
    img = cv2.imread(data_dir + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cr_img = crop_resize(img, new_row, new_col)
    image = np.array(cr_img)
    return image, steering

In [13]:
def generate_train_batch(log, batch_size = 128, filter_ratio = 0.5):

    batch_images = np.zeros((batch_size, new_row, new_col, 3), dtype = np.uint8)
    batch_steering = np.zeros(batch_size, dtype = np.uint8)
    
    while 1:
        for i_batch in range(batch_size):
            
            reselect = 1
            while reselect == 1:
                i_entry = np.random.randint(len(log))
                #print(i_entry)
                entry = log.iloc[i_entry, :]
                image, steering = preprocess_train(entry)
                if abs(steering) < .15 and np.random.uniform() < filter_ratio:
                    reselect = 1
                else:
                    reselect = 0
            batch_images[i_batch, ...] = image
            batch_steering[i_batch, ...] = steering
            
        yield batch_images, batch_steering

In [14]:
def generate_valid(log):

    while 1:
        for i_entry in range(len(log)):
            #print(i_entry)
            entry = log.iloc[i_entry, :]
            image, steering = preprocess_valid(entry)
            #print(image.shape, steering)
            image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
            steering = np.array([[steering]])
            yield image, steering

In [15]:
#valid_generator = generate_valid(driving_log)
#train_generator = generate_train_batch(driving_log, batch_size = 3)

In [16]:
#batch_imgs, batch_strs = next(train_generator)

In [17]:
#fig = plt.figure(figsize = (16, 12))
#for i in range(1):
#    img, steer = next(train_generator)
#    for j in range(3):
#        ax = fig.add_subplot(3, 2, i*3 + j + 1)
#        print('plot shape', img[j,...].shape)
#        ax.imshow(img[j,...].squeeze())
#        #ax.imshow(img[j,...])
#        ax.set_title(str(np.round(steer[j],4)));

In [18]:
# fig = plt.figure(figsize = (16, 12))
# for i in range(6):
#     #next(valid_generator)
#     img, steer = next(valid_generator)
#     ax = fig.add_subplot(3, 2, i + 1)
#     print(img.shape)
#     ax.imshow(img)
        
#     ax.set_title(str(np.round(steer,4)));

In [19]:
# fig = plt.figure(figsize = (16, 12))
# for i in range(5092,5098):
#     #for p in ['left', 'center', 'right']:
#     image, steering = preprocess_train(driving_log.iloc[i,:])
#     print(image.shape)
#     ax = fig.add_subplot(3, 2, i -5091)
#     ax.imshow(image)
#     #ax.set_title('steer =' + str(np.round(steering, 4)))
#     ax.set_title(str(i) + '_steer =' + str(np.round(steering, 4)))    

In [20]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [21]:
model = Sequential()
model.add(Lambda(lambda x : x/255. - 0.5, input_shape = (new_row, new_col, 3)))
# 3 @ 1x1 filter to choose color space automatically
model.add(Convolution2D(3, 1, 1, init = 'he_normal', activation = 'elu', border_mode = 'valid', name = 'conv0'))
#
model.add(Convolution2D(24, 5, 5, init = 'he_normal', subsample = (2, 2), border_mode = 'valid', activation = 'elu', name = 'conv1'))
model.add(Convolution2D(36, 5, 5, init = 'he_normal', subsample = (2, 2), border_mode = 'valid', activation = 'elu', name = 'conv2'))
model.add(Convolution2D(48, 5, 5, init = 'he_normal', subsample = (2, 2), border_mode = 'valid', activation = 'elu', name = 'conv3'))
model.add(Convolution2D(64, 3, 3, init = 'he_normal', subsample = (1, 1), border_mode = 'valid', activation = 'elu', name = 'conv4'))
model.add(Convolution2D(64, 3, 3, init = 'he_normal', subsample = (1, 1), border_mode = 'valid', activation = 'elu', name = 'conv5'))
model.add(Flatten())
model.add(Dense(100, init = 'he_normal', activation = 'elu', name = 'full1'))
model.add(Dense(50, init = 'he_normal', activation = 'elu', name = 'full2'))
model.add(Dense(10, init = 'he_normal', activation = 'elu', name = 'full3'))
model.add(Dense(1, init = 'he_normal', name = 'full4'))

In [22]:
model.compile(loss = 'mse', optimizer = 'adam')

In [26]:
train_generator = generate_train_batch(driving_log, batch_size = 64)
valid_generator = generate_valid(driving_log)

history = model.fit_generator(train_generator,
            samples_per_epoch=8192, nb_epoch=1,validation_data=valid_generator,
                        nb_val_samples=n_entries, verbose = 1)

val_loss = history.history['val_loss'][0]
model.save('model.h5')

Epoch 1/1
8192/8192 [==============================] - 182s - loss: 15.9722 - val_loss: 0.0448
